# 2回
https://sites.google.com/view/keisemi-ebiz/%E7%AC%AC2%E5%9B%9E

In [1]:
import pandas as pd
import numpy as np

In [2]:
car = pd.read_csv("CleanData_20180222.csv", encoding= "shift-jis")
HH = pd.read_csv('HHsize.csv',thousands=',')
CPI = pd.read_csv('zni2015s.csv',encoding="shift-jis")

In [3]:
car = car.rename(columns = {"Year":"year"})
car = car[["Maker","Type","Name","year","Sales","Model","price","kata","weight","FuelEfficiency","HorsePower","overall_length","overall_width","overall_height"]]
car.head(3)

,Maker,Type,Name,year,Sales,Model,price,kata,weight,FuelEfficiency,HorsePower,overall_length,overall_width,overall_height
0,Audi,Foreign,A1シリーズ,2011,4206,1.4 TFSI,289.0,DBA-8XCAX,1190,19.4,122.0,3970,1740,1440
1,Audi,Foreign,A1シリーズ,2012,4502,1.4 TFSI,273.0,DBA-8XCAX,1190,19.4,122.0,3970,1740,1440
2,Audi,Foreign,A1シリーズ,2013,5071,1.4 TFSI,273.0,DBA-8XCAX,1190,19.4,122.0,3970,1740,1440


In [4]:
CPI = CPI[5:]
CPI = CPI.rename(columns = {"類・品目":"year", "総合":"cpi"})
CPI = CPI[["year","cpi"]]
CPI = CPI.astype({"year":int,"cpi":float})
CPI = CPI.reset_index(drop= True)
CPI.head(3)

,year,cpi
0,1970,31.5
1,1971,33.5
2,1972,35.2


In [5]:
# 3つのデータをyearを引数にmergeする．
data = pd.merge(car,HH,on = "year",how = "left")
data = pd.merge(data, CPI,on = "year",how = "left")
data.head(3)

,Maker,Type,Name,year,Sales,Model,price,kata,weight,FuelEfficiency,HorsePower,overall_length,overall_width,overall_height,HH,cpi
0,Audi,Foreign,A1シリーズ,2011,4206,1.4 TFSI,289.0,DBA-8XCAX,1190,19.4,122.0,3970,1740,1440,53783435,96.3
1,Audi,Foreign,A1シリーズ,2012,4502,1.4 TFSI,273.0,DBA-8XCAX,1190,19.4,122.0,3970,1740,1440,54171475,96.2
2,Audi,Foreign,A1シリーズ,2013,5071,1.4 TFSI,273.0,DBA-8XCAX,1190,19.4,122.0,3970,1740,1440,54594744,96.6


In [6]:
# 燃費が欠損しているデータを落とす．
data = data.query('FuelEfficiency.notna()', engine='python')

In [7]:
# 価格の実質化 2016年が基準
cpi2016 = data.query("year == 2016")["cpi"].iloc[0]

data["price"] = (data["price"]/(data["cpi"]/cpi2016))
data["price"] =  data["price"]/100

In [8]:
# サイズ，燃費の重量に対する比率を定義
data["size"] = (data["overall_length"]/1000)*(data["overall_width"]/1000)*(data["overall_height"]/1000)
data["hppw"] = data["HorsePower"]/data["weight"]

In [9]:
# 自動車の車種IDを割り当て
data["NameID"] = pd.factorize(data["Name"])[0]

In [10]:
# 変数の準備
data['inside_total'] = data.groupby('year')['Sales'].transform('sum')
data.head()

,Maker,Type,Name,year,Sales,Model,price,kata,weight,FuelEfficiency,HorsePower,overall_length,overall_width,overall_height,HH,cpi,size,hppw,NameID,inside_total
0,Audi,Foreign,A1シリーズ,2011,4206,1.4 TFSI,2.998037,DBA-8XCAX,1190,19.4,122.0,3970,1740,1440,53783435,96.3,9.947232,0.102521,0,3281806
1,Audi,Foreign,A1シリーズ,2012,4502,1.4 TFSI,2.835000,DBA-8XCAX,1190,19.4,122.0,3970,1740,1440,54171475,96.2,9.947232,0.102521,0,4346619
2,Audi,Foreign,A1シリーズ,2013,5071,1.4 TFSI,2.823261,DBA-8XCAX,1190,19.4,122.0,3970,1740,1440,54594744,96.6,9.947232,0.102521,0,4391427
3,Audi,Foreign,A3シリーズ,2006,4830,アトラクション,2.918889,GH-8PBSE,1360,12.2,102.0,4285,1765,1430,51102005,97.2,10.815126,0.075000,1,4332043
4,Audi,Foreign,A3シリーズ,2007,3874,アトラクション,2.939444,GH-8PBSE,1360,12.2,102.0,4285,1765,1430,51713048,97.2,10.815126,0.075000,1,4128036


In [11]:
# 変数を定義
data["outside_total"] = data["HH"]-data["inside_total"]
data["share"] = data["Sales"]/data["HH"]
data["share0"] = data["outside_total"]/data["HH"]

In [12]:
# 必要な列だけ
data = data[["NameID","year","Maker","Type","Sales","price","FuelEfficiency","HH","size","hppw","share","share0"]]
data

,NameID,year,Maker,Type,Sales,price,FuelEfficiency,HH,size,hppw,share,share0
0,0,2011,Audi,Foreign,4206,2.998037,19.4,53783435,9.947232,0.102521,0.000078,0.938981
1,0,2012,Audi,Foreign,4502,2.835000,19.4,54171475,9.947232,0.102521,0.000083,0.919762
2,0,2013,Audi,Foreign,5071,2.823261,19.4,54594744,9.947232,0.102521,0.000093,0.919563
3,1,2006,Audi,Foreign,4830,2.918889,12.2,51102005,10.815126,0.075000,0.000095,0.915228
4,1,2007,Audi,Foreign,3874,2.939444,12.2,51713048,10.815126,0.075000,0.000075,0.920174
...,...,...,...,...,...,...,...,...,...,...,...,...
1956,259,2006,Volvo,Foreign,5990,4.717500,9.6,51102005,12.694204,0.108974,0.000117,0.915228
1957,259,2007,Volvo,Foreign,6018,4.666111,9.6,51713048,12.593196,0.108974,0.000116,0.920174
1958,259,2008,Volvo,Foreign,3635,4.944341,8.6,52324877,14.089241,0.114286,0.000069,0.923564
1959,259,2009,Volvo,Foreign,2006,5.128611,8.6,52877802,14.089241,0.114286,0.000038,0.929190


## 3.3 操作変数の構築

*   製品の属性
    1. 重量に対する馬力(hppw)
    2. 燃費(FuelEfficiency)
    3. 大きさ(size)

### BLP1
* 同じ企業が生産している，他の製品の製品品質の和をつくる．
  1. 企業と年(マーケット$t$)でgroupby.sum()
  2. 元データに，企業と年を引数に，左からマージ
  3. group.sum()から行の製品品質のデータを除いて，BLP1の完成．

### BLP2
* 他の企業が生産している．製品の品質の和をつくる．
  1. 年(マーケット$t$)でgroupby.sum()
  2. 元データに，年を引数に，左からマージ
  3. group.sum()からBLP1のgroup.sum()を除いて，BLP1の完成．


In [13]:
data[["year","Maker","hppw","FuelEfficiency","size"]].groupby(["year","Maker"]).sum()

hppw  FuelEfficiency        size
year Maker                                           
2006 Audi        0.163435            23.4   22.420219
     BMW         0.396802            43.6   44.661924
     Daihatsu    0.943049           215.0  107.161481
     Honda       2.224907           325.6  234.986479
     Lexas       0.688389            39.6   49.301213
...                   ...             ...         ...
2016 Subaru      1.298413           243.2  142.993741
     Suzuki      1.173852           302.0  137.521509
     Toyota      3.842385           721.0  499.764264
     Volkswagen  0.330407            78.4   45.639509
     Volvo       0.195386            33.0   23.983371

[163 rows x 3 columns]

## BLP その1

In [14]:
# Step1
BLP1 = data.groupby(["year","Maker"]).agg(hppw_own_sum = ("hppw","sum"),
                                          FuelEfficiency_own_sum	= ("FuelEfficiency","sum"),
                                          size_own_sum = ("size","sum"))
BLP1 = BLP1.reset_index()
BLP1.head()

,year,Maker,hppw_own_sum,FuelEfficiency_own_sum,size_own_sum
0,2006,Audi,0.163435,23.4,22.420219
1,2006,BMW,0.396802,43.6,44.661924
2,2006,Daihatsu,0.943049,215.0,107.161481
3,2006,Honda,2.224907,325.6,234.986479
4,2006,Lexas,0.688389,39.6,49.301213


In [15]:
# Step2
data = pd.merge(data,BLP1, on = ["year","Maker"], how = "left")
data.head()

,NameID,year,Maker,Type,Sales,price,FuelEfficiency,HH,size,hppw,share,share0,hppw_own_sum,FuelEfficiency_own_sum,size_own_sum
0,0,2011,Audi,Foreign,4206,2.998037,19.4,53783435,9.947232,0.102521,0.000078,0.938981,0.299061,49.4,33.139717
1,0,2012,Audi,Foreign,4502,2.835000,19.4,54171475,9.947232,0.102521,0.000083,0.919762,0.309984,49.8,33.179137
2,0,2013,Audi,Foreign,5071,2.823261,19.4,54594744,9.947232,0.102521,0.000093,0.919563,0.311828,52.7,33.545573
3,1,2006,Audi,Foreign,4830,2.918889,12.2,51102005,10.815126,0.075000,0.000095,0.915228,0.163435,23.4,22.420219
4,1,2007,Audi,Foreign,3874,2.939444,12.2,51713048,10.815126,0.075000,0.000075,0.920174,0.312689,35.4,33.110987


In [16]:
# Step3
data["iv_BLP_own_hppw"] = data["hppw_own_sum"] - data["hppw"]
data["iv_BLP_own_FuelEfficiency"] = data["FuelEfficiency_own_sum"] - data["FuelEfficiency"]
data["iv_BLP_own_size"] = data["size_own_sum"] - data["size"]
data

,NameID,year,Maker,Type,Sales,price,FuelEfficiency,HH,size,hppw,share,share0,hppw_own_sum,FuelEfficiency_own_sum,size_own_sum,iv_BLP_own_hppw,iv_BLP_own_FuelEfficiency,iv_BLP_own_size
0,0,2011,Audi,Foreign,4206,2.998037,19.4,53783435,9.947232,0.102521,0.000078,0.938981,0.299061,49.4,33.139717,0.196540,30.0,23.192485
1,0,2012,Audi,Foreign,4502,2.835000,19.4,54171475,9.947232,0.102521,0.000083,0.919762,0.309984,49.8,33.179137,0.207463,30.4,23.231905
2,0,2013,Audi,Foreign,5071,2.823261,19.4,54594744,9.947232,0.102521,0.000093,0.919563,0.311828,52.7,33.545573,0.209307,33.3,23.598341
3,1,2006,Audi,Foreign,4830,2.918889,12.2,51102005,10.815126,0.075000,0.000095,0.915228,0.163435,23.4,22.420219,0.088435,11.2,11.605093
4,1,2007,Audi,Foreign,3874,2.939444,12.2,51713048,10.815126,0.075000,0.000075,0.920174,0.312689,35.4,33.110987,0.237689,23.2,22.295861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1818,259,2006,Volvo,Foreign,5990,4.717500,9.6,51102005,12.694204,0.108974,0.000117,0.915228,0.108974,9.6,12.694204,0.000000,0.0,0.000000
1819,259,2007,Volvo,Foreign,6018,4.666111,9.6,51713048,12.593196,0.108974,0.000116,0.920174,0.108974,9.6,12.593196,0.000000,0.0,0.000000
1820,259,2008,Volvo,Foreign,3635,4.944341,8.6,52324877,14.089241,0.114286,0.000069,0.923564,0.114286,8.6,14.089241,0.000000,0.0,0.000000
1821,259,2009,Volvo,Foreign,2006,5.128611,8.6,52877802,14.089241,0.114286,0.000038,0.929190,0.215684,20.2,25.689821,0.101399,11.6,11.600580


## BLP その2

In [17]:
# Step1
BLP2 = data.groupby(["year"]).agg(hppw_other_sum = ("hppw","sum"),
                                  FuelEfficiency_other_sum = ("FuelEfficiency","sum"),
                                  size_other_sum = ("size","sum"))
BLP2 = BLP2.reset_index()
BLP2.head()

,year,hppw_other_sum,FuelEfficiency_other_sum,size_other_sum
0,2006,16.324308,2414.4,1863.322858
1,2007,16.366041,2417.4,1872.269921
2,2008,16.373780,2448.6,1872.271369
3,2009,16.697093,2451.2,1912.187651
4,2010,17.143632,2569.6,1952.225655


In [18]:
# Step2
data = pd.merge(data,BLP2, on = ["year"], how = "left")
data.head()

,NameID,year,Maker,Type,Sales,price,FuelEfficiency,HH,size,hppw,...,share0,hppw_own_sum,FuelEfficiency_own_sum,size_own_sum,iv_BLP_own_hppw,iv_BLP_own_FuelEfficiency,iv_BLP_own_size,hppw_other_sum,FuelEfficiency_other_sum,size_other_sum
0,0,2011,Audi,Foreign,4206,2.998037,19.4,53783435,9.947232,0.102521,...,0.938981,0.299061,49.4,33.139717,0.196540,30.0,23.192485,16.021958,2513.9,1844.750029
1,0,2012,Audi,Foreign,4502,2.835000,19.4,54171475,9.947232,0.102521,...,0.919762,0.309984,49.8,33.179137,0.207463,30.4,23.231905,16.495105,2709.1,1924.543923
2,0,2013,Audi,Foreign,5071,2.823261,19.4,54594744,9.947232,0.102521,...,0.919563,0.311828,52.7,33.545573,0.209307,33.3,23.598341,15.850303,2814.9,1891.172852
3,1,2006,Audi,Foreign,4830,2.918889,12.2,51102005,10.815126,0.075000,...,0.915228,0.163435,23.4,22.420219,0.088435,11.2,11.605093,16.324308,2414.4,1863.322858
4,1,2007,Audi,Foreign,3874,2.939444,12.2,51713048,10.815126,0.075000,...,0.920174,0.312689,35.4,33.110987,0.237689,23.2,22.295861,16.366041,2417.4,1872.269921


In [19]:
# Step3
data["iv_BLP_other_hppw"] = data["hppw_other_sum"] - data["hppw_own_sum"]
data["iv_BLP_other_FuelEfficiency"] = data["FuelEfficiency_other_sum"] - data["FuelEfficiency_own_sum"]
data["iv_BLP_other_size"] = data["size_other_sum"] - data["size_own_sum"]
data.head()

,NameID,year,Maker,Type,Sales,price,FuelEfficiency,HH,size,hppw,...,size_own_sum,iv_BLP_own_hppw,iv_BLP_own_FuelEfficiency,iv_BLP_own_size,hppw_other_sum,FuelEfficiency_other_sum,size_other_sum,iv_BLP_other_hppw,iv_BLP_other_FuelEfficiency,iv_BLP_other_size
0,0,2011,Audi,Foreign,4206,2.998037,19.4,53783435,9.947232,0.102521,...,33.139717,0.196540,30.0,23.192485,16.021958,2513.9,1844.750029,15.722897,2464.5,1811.610312
1,0,2012,Audi,Foreign,4502,2.835000,19.4,54171475,9.947232,0.102521,...,33.179137,0.207463,30.4,23.231905,16.495105,2709.1,1924.543923,16.185121,2659.3,1891.364786
2,0,2013,Audi,Foreign,5071,2.823261,19.4,54594744,9.947232,0.102521,...,33.545573,0.209307,33.3,23.598341,15.850303,2814.9,1891.172852,15.538474,2762.2,1857.627279
3,1,2006,Audi,Foreign,4830,2.918889,12.2,51102005,10.815126,0.075000,...,22.420219,0.088435,11.2,11.605093,16.324308,2414.4,1863.322858,16.160873,2391.0,1840.902638
4,1,2007,Audi,Foreign,3874,2.939444,12.2,51713048,10.815126,0.075000,...,33.110987,0.237689,23.2,22.295861,16.366041,2417.4,1872.269921,16.053352,2382.0,1839.158934


## Differentiation IV (Gandhi-Houde)
1. GH1
* 同じ企業が生産している，他の製品の製品品質の和をつくる．
  0. 品質列の2乗を作成
  1. 企業と年(マーケット t )でgroupby.sum()とgoupt.count()
  2. 元データに，企業と年を引数に，左からマージ
  3. group.sum()から行の製品品質のデータを除いて，BLP1の完成．
2. GH2
* 他の企業が生産している．製品の品質の和をつくる．
  1. 年(マーケット t )でgroupby.sum()
  2. 元データに，年を引数に，左からマージ
  3. group.sum()からBLP1のgroup.sum()を除いて，BLP1の完成．

In [20]:
data[["year","Maker","hppw","FuelEfficiency","size"]]

,year,Maker,hppw,FuelEfficiency,size
0,2011,Audi,0.102521,19.4,9.947232
1,2012,Audi,0.102521,19.4,9.947232
2,2013,Audi,0.102521,19.4,9.947232
3,2006,Audi,0.075000,12.2,10.815126
4,2007,Audi,0.075000,12.2,10.815126
...,...,...,...,...,...
1818,2006,Volvo,0.108974,9.6,12.694204
1819,2007,Volvo,0.108974,9.6,12.593196
1820,2008,Volvo,0.114286,8.6,14.089241
1821,2009,Volvo,0.114286,8.6,14.089241


In [21]:
# Step0
data["hppw_sqr"] = data["hppw"]**2
data["FuelEfficiency_sqr"] = data["FuelEfficiency"]**2
data["size_sqr"] = data["size"]**2

In [22]:
# Step1
GH1 = (data.groupby(['year', 'Maker'])
    .agg(hppw_sqr_own_sum=('hppw_sqr', 'sum'),
         FuelEfficiency_sqr_own_sum=('FuelEfficiency_sqr', 'sum'),
         size_sqr_own_sum=('size_sqr', 'sum'),
         count_GH1 = ('hppw',"count")
    )
)
GH1 = GH1.reset_index()
GH1

,year,Maker,hppw_sqr_own_sum,FuelEfficiency_sqr_own_sum,size_sqr_own_sum,count_GH1
0,2006,Audi,0.013446,274.28,251.645140,2
1,2006,BMW,0.041196,481.20,508.337602,4
2,2006,Daihatsu,0.076238,4010.92,997.335507,12
3,2006,Honda,0.269372,5612.96,2726.536322,21
4,2006,Lexas,0.120924,397.74,611.490026,4
...,...,...,...,...,...,...
158,2016,Subaru,0.149268,5009.52,1628.709053,13
159,2016,Suzuki,0.103452,7149.44,1405.368298,14
160,2016,Toyota,0.388206,15287.38,6520.212118,40
161,2016,Volkswagen,0.027316,1553.36,525.783571,4


In [23]:
# Step2
data = pd.merge(data,GH1, on = ["year","Maker"], how = "left")
data.head()

,NameID,year,Maker,Type,Sales,price,FuelEfficiency,HH,size,hppw,...,iv_BLP_other_hppw,iv_BLP_other_FuelEfficiency,iv_BLP_other_size,hppw_sqr,FuelEfficiency_sqr,size_sqr,hppw_sqr_own_sum,FuelEfficiency_sqr_own_sum,size_sqr_own_sum,count_GH1
0,0,2011,Audi,Foreign,4206,2.998037,19.4,53783435,9.947232,0.102521,...,15.722897,2464.5,1811.610312,0.010511,376.36,98.947424,0.029943,831.48,369.074292,3
1,0,2012,Audi,Foreign,4502,2.835000,19.4,54171475,9.947232,0.102521,...,16.185121,2659.3,1891.364786,0.010511,376.36,98.947424,0.032377,842.36,370.050682,3
2,0,2013,Audi,Foreign,5071,2.823261,19.4,54594744,9.947232,0.102521,...,15.538474,2762.2,1857.627279,0.010511,376.36,98.947424,0.032714,947.05,378.120304,3
3,1,2006,Audi,Foreign,4830,2.918889,12.2,51102005,10.815126,0.075000,...,16.160873,2391.0,1840.902638,0.005625,148.84,116.966945,0.013446,274.28,251.645140,2
4,1,2007,Audi,Foreign,3874,2.939444,12.2,51713048,10.815126,0.075000,...,16.053352,2382.0,1839.158934,0.005625,148.84,116.966945,0.035722,418.28,365.937661,3


In [24]:
# Step3
data["iv_GH_own_hppw"] = (data["count_GH1"]-1)*data["hppw_sqr"] + (data["hppw_sqr_own_sum"] - data["hppw_sqr"]) - 2*data["hppw"]*(data["hppw_own_sum"] - data["hppw"])
data["iv_GH_own_FuelEfficiency"] = (data["count_GH1"]-1)*data["FuelEfficiency_sqr"] + (data["FuelEfficiency_sqr_own_sum"] - data["FuelEfficiency_sqr"]) - 2*data["FuelEfficiency"]*(data["FuelEfficiency_own_sum"] - data["FuelEfficiency"])
data["iv_GH_own_size"] = (data["count_GH1"]-1)*data["size_sqr"] + (data["size_sqr_own_sum"] - data["size_sqr"]) - 2*data["size"]*(data["size_own_sum"] - data["size"])
data

,NameID,year,Maker,Type,Sales,price,FuelEfficiency,HH,size,hppw,...,hppw_sqr,FuelEfficiency_sqr,size_sqr,hppw_sqr_own_sum,FuelEfficiency_sqr_own_sum,size_sqr_own_sum,count_GH1,iv_GH_own_hppw,iv_GH_own_FuelEfficiency,iv_GH_own_size
0,0,2011,Audi,Foreign,4206,2.998037,19.4,53783435,9.947232,0.102521,...,0.010511,376.36,98.947424,0.029943,831.48,369.074292,3,0.000154,43.84,6.619649
1,0,2012,Audi,Foreign,4502,2.835000,19.4,54171475,9.947232,0.102521,...,0.010511,376.36,98.947424,0.032377,842.36,370.050682,3,0.000349,39.20,6.811799
2,0,2013,Audi,Foreign,5071,2.823261,19.4,54594744,9.947232,0.102521,...,0.010511,376.36,98.947424,0.032714,947.05,378.120304,3,0.000308,31.37,7.591378
3,1,2006,Audi,Foreign,4830,2.918889,12.2,51102005,10.815126,0.075000,...,0.005625,148.84,116.966945,0.013446,274.28,251.645140,2,0.000181,1.00,0.624049
4,1,2007,Audi,Foreign,3874,2.939444,12.2,51713048,10.815126,0.075000,...,0.005625,148.84,116.966945,0.035722,418.28,365.937661,3,0.005694,1.04,0.639514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1818,259,2006,Volvo,Foreign,5990,4.717500,9.6,51102005,12.694204,0.108974,...,0.011875,92.16,161.142815,0.011875,92.16,161.142815,1,0.000000,0.00,0.000000
1819,259,2007,Volvo,Foreign,6018,4.666111,9.6,51713048,12.593196,0.108974,...,0.011875,92.16,158.588585,0.011875,92.16,158.588585,1,0.000000,0.00,0.000000
1820,259,2008,Volvo,Foreign,3635,4.944341,8.6,52324877,14.089241,0.114286,...,0.013061,73.96,198.506719,0.013061,73.96,198.506719,1,0.000000,0.00,0.000000
1821,259,2009,Volvo,Foreign,2006,5.128611,8.6,52877802,14.089241,0.114286,...,0.013061,73.96,198.506719,0.023343,208.52,333.080175,2,0.000166,9.00,6.193435


In [25]:
# GH2
# Step1
GH2 = (
    data.groupby('year')
    .agg(
        hppw_sqr_other_sum=('hppw_sqr', 'sum'),
        FuelEfficiency_sqr_other_sum=('FuelEfficiency_sqr', 'sum'),
        size_sqr_other_sum=('size_sqr', 'sum'),
        count_GH2=('hppw', 'count')
    )
)
GH2 = GH2.reset_index()
GH2

,year,hppw_sqr_other_sum,FuelEfficiency_sqr_other_sum,size_sqr_other_sum,count_GH2
0,2006,1.776015,38911.12,22009.511368,165
1,2007,1.785286,38762.92,22237.876797,165
2,2008,1.835727,39436.96,22275.939131,165
3,2009,1.894772,39451.98,23050.604158,166
4,2010,1.946219,42192.38,23463.329847,170
5,2011,1.819955,43068.89,22207.674377,160
6,2012,1.847895,48757.65,23198.867591,167
7,2013,1.728182,54138.71,22804.643189,164
8,2014,1.849025,57529.84,23601.028556,167
9,2015,1.833052,62351.38,23541.060479,165


In [26]:
# Step2
data = pd.merge(data,GH2, on = ["year"], how = "left")
data.head()

,NameID,year,Maker,Type,Sales,price,FuelEfficiency,HH,size,hppw,...,FuelEfficiency_sqr_own_sum,size_sqr_own_sum,count_GH1,iv_GH_own_hppw,iv_GH_own_FuelEfficiency,iv_GH_own_size,hppw_sqr_other_sum,FuelEfficiency_sqr_other_sum,size_sqr_other_sum,count_GH2
0,0,2011,Audi,Foreign,4206,2.998037,19.4,53783435,9.947232,0.102521,...,831.48,369.074292,3,0.000154,43.84,6.619649,1.819955,43068.89,22207.674377,160
1,0,2012,Audi,Foreign,4502,2.835000,19.4,54171475,9.947232,0.102521,...,842.36,370.050682,3,0.000349,39.20,6.811799,1.847895,48757.65,23198.867591,167
2,0,2013,Audi,Foreign,5071,2.823261,19.4,54594744,9.947232,0.102521,...,947.05,378.120304,3,0.000308,31.37,7.591378,1.728182,54138.71,22804.643189,164
3,1,2006,Audi,Foreign,4830,2.918889,12.2,51102005,10.815126,0.075000,...,274.28,251.645140,2,0.000181,1.00,0.624049,1.776015,38911.12,22009.511368,165
4,1,2007,Audi,Foreign,3874,2.939444,12.2,51713048,10.815126,0.075000,...,418.28,365.937661,3,0.005694,1.04,0.639514,1.785286,38762.92,22237.876797,165


In [27]:
# Step3
data["iv_GH_other_hppw"] = (data["count_GH2"]-data["count_GH1"])*data["hppw_sqr"] + (data["hppw_sqr_other_sum"] - data["hppw_sqr_own_sum"]) - 2*data["hppw"]*(data["hppw_other_sum"] - data["hppw_own_sum"])
data["iv_GH_other_FuelEfficiency"] = (data["count_GH2"]-data["count_GH1"])*data["FuelEfficiency_sqr"] + (data["FuelEfficiency_sqr_other_sum"] - data["FuelEfficiency_sqr_own_sum"]) - 2*data["FuelEfficiency"]*(data["FuelEfficiency_other_sum"] - data["FuelEfficiency_own_sum"])
data["iv_GH_other_size"] = (data["count_GH2"]-data["count_GH1"])*data["size_sqr"] + (data["size_sqr_other_sum"] - data["size_sqr_own_sum"]) - 2*data["size"]*(data["size_other_sum"] - data["size_own_sum"])
data

,NameID,year,Maker,Type,Sales,price,FuelEfficiency,HH,size,hppw,...,iv_GH_own_hppw,iv_GH_own_FuelEfficiency,iv_GH_own_size,hppw_sqr_other_sum,FuelEfficiency_sqr_other_sum,size_sqr_other_sum,count_GH2,iv_GH_other_hppw,iv_GH_other_FuelEfficiency,iv_GH_other_size
0,0,2011,Audi,Foreign,4206,2.998037,19.4,53783435,9.947232,0.102521,...,0.000154,43.84,6.619649,1.819955,43068.89,22207.674377,160,0.216315,5703.33,1332.329593
1,0,2012,Audi,Foreign,4502,2.835000,19.4,54171475,9.947232,0.102521,...,0.000349,39.20,6.811799,1.847895,48757.65,23198.867591,167,0.220620,6457.49,1428.505882
2,0,2013,Audi,Foreign,5071,2.823261,19.4,54594744,9.947232,0.102521,...,0.000308,31.37,7.591378,1.728182,54138.71,22804.643189,164,0.201628,6612.26,1400.559199
3,1,2006,Audi,Foreign,4830,2.918889,12.2,51102005,10.815126,0.075000,...,0.000181,1.00,0.624049,1.776015,38911.12,22009.511368,165,0.255313,4557.36,1004.291203
4,1,2007,Audi,Foreign,3874,2.939444,12.2,51713048,10.815126,0.075000,...,0.005694,1.04,0.639514,1.785286,38762.92,22237.876797,165,0.252811,4335.92,1039.113937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1818,259,2006,Volvo,Foreign,5990,4.717500,9.6,51102005,12.694204,0.108974,...,0.000000,0.00,0.000000,1.776015,38911.12,22009.511368,165,0.177596,7761.04,1291.274926
1819,259,2007,Volvo,Foreign,6018,4.666111,9.6,51713048,12.593196,0.108974,...,0.000000,0.00,0.000000,1.785286,38762.92,22237.876797,165,0.177771,7555.24,1249.269241
1820,259,2008,Volvo,Foreign,3635,4.944341,8.6,52324877,14.089241,0.114286,...,0.000000,0.00,0.000000,1.835727,39436.96,22275.939131,165,0.248251,9524.44,2271.781773
1821,259,2009,Volvo,Foreign,2006,5.128611,8.6,52877802,14.089241,0.114286,...,0.000166,9.00,6.193435,1.894772,39451.98,23050.604158,166,0.246291,9559.70,2113.979831


In [28]:
# 変数の作成
# 対数
#data["log_sales"] = data["Sales"].apply(np.log)
#data["log_price"] = data["price"].apply(np.log)
data["logit_share"] = data["share"].apply(np.log) - data["share0"].apply(np.log)


## データフレームのアウトプット

In [29]:
# 変数を除く前のデータ
# data.to_csv("data_2_raw.csv",index = False)

In [30]:
# 分析に使用するデータのみ抽出
data = data.drop(columns = {"Maker","Type"})
words_ends = ("_sqr_own_sum","_sqr_other_sum","_other_sum","_sqr","_GH1","_GH2")

# words_endの条件を満たさないデータを抽出
data = data.loc[:, ~data.columns.str.endswith(words_ends)]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1823 entries, 0 to 1822
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   NameID                       1823 non-null   int64  
 1   year                         1823 non-null   int64  
 2   Sales                        1823 non-null   int64  
 3   price                        1823 non-null   float64
 4   FuelEfficiency               1823 non-null   float64
 5   HH                           1823 non-null   int64  
 6   size                         1823 non-null   float64
 7   hppw                         1823 non-null   float64
 8   share                        1823 non-null   float64
 9   share0                       1823 non-null   float64
 10  hppw_own_sum                 1823 non-null   float64
 11  FuelEfficiency_own_sum       1823 non-null   float64
 12  size_own_sum                 1823 non-null   float64
 13  iv_BLP_own_hppw   

In [31]:
# csvに書き出し
data.to_csv("data_2.csv", index = False)

In [32]:
import random

# サンプルを抽出
NameID = data["NameID"].unique()
NameID = NameID.tolist()

random.seed(123)
NameID_NIPYO = random.sample(NameID, 30)

data_NIPYO = data.query("NameID == @NameID_NIPYO")

# csvに書き出し
data_NIPYO.to_csv("data_2_NIPYO.csv", index = False)